In [1]:
import time
import matplotlib.pyplot as plt
import numpy as np
import math as mt
import seaborn as sns
from tqdm import tqdm
import pandas as pd
from konlpy.tag import Hannanum, Mecab
from hangul_utils import split_syllables, join_jamos
from tqdm.auto import tqdm
from kiwipiepy import Kiwi
import re
from pykospacing import Spacing
#if you use window, import eungeon instand of konlpy
#from eunjeon import Mecab

#import sys
#import io
#sys.stdout = io.TextIOWrapper(sys.stdout.detach(), encoding = 'utf-8')
#sys.stderr = io.TextIOWrapper(sys.stderr.detach(), encoding = 'utf-8')

han = Hannanum()
mec = Mecab()

number = 100

# 초성 리스트. 00 ~ 18
CHOSUNG_LIST = ['ㄱ', 'ㄲ', 'ㄴ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅃ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']
# 중성 리스트. 00 ~ 20
JUNGSUNG_LIST = ['ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ', 'ㅗ', 'ㅘ', 'ㅙ', 'ㅚ', 'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 'ㅡ', 'ㅢ', 'ㅣ']
# 종성 리스트. 00 ~ 27 + 1(1개 없음)
JONGSUNG_LIST = ['_', 'ㄱ', 'ㄲ', 'ㄳ', 'ㄴ', 'ㄵ', 'ㄶ', 'ㄷ', 'ㄹ', 'ㄺ', 'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 'ㅁ', 'ㅂ', 'ㅄ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']

##높임말 리스트. 다음에 해당하면 높임말로 판단한다.
H_LIST = ['니다', '니까', '요', '시오', '죠']

##높임말로 변환할 때 중성끼리 합치는 경우가 있다. 그러한 경우 합치기 위한 사전이다. 
con_dict = [
    
    ['ㅏㅣ','ㅐ'], ['ㅑㅣ','ㅒ'], ['ㅓㅣ','ㅔ'],
    ['ㅕㅣ','ㅖ'], ['ㅗㅣ','ㅚ'], ['ㅗㅐ','ㅙ'],
    ['ㅜㅓ','ㅝ'], ['ㅜㅔ','ㅞ'], ['ㅡㅣ','ㅢ'],
    ['ㅣㅏ','ㅑ'], ['ㅣㅓ','ㅕ'], ['ㅣㅗ','ㅛ'],
    ['ㅣㅜ','ㅠ'], ['ㅡㅓ','ㅓ']
    
]

####high -> low Dictionary####

##EF(종결어미): 형태소 분류 시 종결어미가 단독으로 분석 되는 경우
EF_ONLY = [
    
    [['ㅅ','ㅡ','ㅂ','ㄴ','ㅣ','ㄷ','ㅏ'],['ㄷ','ㅏ']], #ex)'습니다'-> '다'
    [['ㄴ','ㅏ','ㅇ','ㅛ'],['ㄴ','ㅏ']],
    [['ㄷ','ㅐ','ㅇ','ㅛ'],['ㄷ','ㅐ']],
    [['ㅇ','ㅔ','ㅇ','ㅛ'],['ㅇ','ㅑ']],
    [['ㄴ','ㅔ','ㅇ','ㅛ'],['ㄷ','ㅏ']],
    [['ㄴ','ㅡ','ㄴ','ㄷ','ㅔ','ㅇ','ㅛ'],['ㄴ','ㅡ','ㄴ','ㄷ','ㅔ']],
    [['ㅈ','ㅛ'],['ㅈ','ㅣ']],
    [['ㅇ','ㅏ','ㅇ','ㅛ'],['ㅇ','ㅏ']],
    [['ㅇ','ㅓ','ㅇ','ㅛ'],['ㅇ','ㅓ']],
    [['ㅇ','ㅡ','ㅅ','ㅔ','ㅇ','ㅛ'],['ㅇ','ㅓ']],
    [['ㄱ','ㅓ','ㄷ','ㅡ','ㄴ','ㅇ','ㅛ'],['ㄱ','ㅓ','ㄷ','ㅡ','ㄴ']],
    #[['ㅎ','ㅅ','ㅡ','ㅂ','ㄴ','ㅣ','ㄲ','ㅏ'],['ㅎ','ㅇ','ㅏ']],
    [['ㅅ','ㅡ','ㅂ','ㄴ','ㅣ','ㄲ','ㅏ'],['ㅇ','ㅓ']],
    [['ㄴ','ㅡ','ㄴ','ㄷ','ㅔ','ㅇ','ㅛ'],['ㄴ','ㅡ','ㄴ','ㄷ','ㅔ']],
    [['ㅇ','ㅛ'],['']],
    
]

#EP(선어말어미)+EF(종결어미)로 형태소가 분석될 경우
EP_EF = [
    
    [['ㅅ','ㅔ','ㅇ','ㅛ'],['special']]
    #-시오 family
    ,[['ㅡ','ㅅ','ㅣ','ㅂ','ㅅ','ㅣ','ㅇ','ㅗ'],['ㅓ']]
    ,[['ㅅ','ㅣ','ㅂ','ㅅ','ㅣ','ㅇ','ㅗ'],['ㄹ', 'ㅏ']],
    [['ㅅ','ㅣ','ㅂ','ㄴ','ㅣ','ㄲ','ㅏ'],['ㄴ','ㅣ']],
    [['ㅅ','ㅣ','ㄹ','ㄹ','ㅐ','ㅇ','ㅛ'],['ㄹ','ㄹ','ㅐ']],
]

#VCP+EF
VCP_EF = [
    
    [['ㅇ','ㅣ','ㅂ','ㄴ','ㅣ','ㄷ','ㅏ'],['ㅇ','ㅣ','ㄷ','ㅏ']],
    [['ㅇ','ㅣ','ㅂ','ㄴ','ㅣ','ㄲ','ㅏ'],['ㅇ','ㅣ','ㅇ','ㅑ']],
    [['ㅈ','ㅛ'],['ㅇ','ㅑ']],
    [['ㅇ','ㅣ','ㅇ','ㅔ','ㅇ','ㅛ'],['ㅇ','ㅣ','ㅇ','ㅑ']],
    [['ㅇ','ㅖ','ㅇ','ㅛ'],['ㅇ','ㅑ']],
    [['ㄱ','ㅓ','ㅇ','ㅛ'],['ㅇ','ㅑ']],
    [['ㄹ','ㅏ','ㄷ','ㅓ','ㄴ','ㄷ','ㅔ','ㅇ','ㅛ'],['ㄹ','ㅏ','ㄷ','ㅓ','ㄴ','ㄷ','ㅔ']],
    
]

#XSA+EF
XSA_EF = [
    [['ㅂ','ㄴ','ㅣ','ㄷ','ㅏ'],['ㄷ','ㅏ']],
    [['ㅇ','ㅛ'],['']]
]

#XSV+EF
XSV_EF = [
    [['ㅂ','ㄴ','ㅣ','ㄷ','ㅏ'],['ㄷ','ㅏ']],
    [['ㅎ','ㅐ','ㅇ','ㅛ'],['ㅎ','ㅏ','ㅈ','ㅏ']],
]

#alpha+EF: 위의 경우를 제외한 나머지 경우를 의미한다. 
A_EF = [
    [['ㄱ','ㅓ','ㅂ','ㄴ','ㅣ','ㄷ','ㅏ'],['ㄷ','ㅏ']],
    [['ㅂ','ㄴ','ㅣ','ㄷ','ㅏ'],['ㄴ','ㄷ','ㅏ']],
    [['ㄱ','ㅖ','ㅅ','ㅔ','ㅇ','ㅛ'],['ㅇ','ㅣ','ㅆ','ㅇ','ㅓ']],
    [['ㅂ','ㄴ','ㅣ','ㄲ','ㅏ'],['ㄴ','ㅏ']],
    [['ㅇ','ㅛ'],['']]
    
    
]

##높임표현이 어말어미가 아닌 경우##
# - 기존의 변환 방식이었다. 이제는 쓰지는 않지만, 코드의 오류가 발생할 가능성이 있기에 지우지 않았다. 따로 참고할 필요 없음.
EF = [
    [['ㅅ','ㅡ','ㅂ','ㄴ','ㅣ','ㄷ','ㅏ'],['ㄷ','ㅏ']],
    [[' ','ㅈ','ㅓ','ㄴ',' '],['ㄴ','ㅏ','_','ㄴ','ㅡ','ㄴ']],
    [['ㅇ','ㅓ','_','ㅇ','ㅛ','_'],['ㄷ','ㅏ','_']],
    [['ㅂ','ㄴ','ㅣ','ㄷ','ㅏ'],['ㄴ','ㄷ','ㅏ']],
    [['ㅇ','ㅔ','ㅇ','ㅛ'],['ㅇ','ㅑ']],
    [['ㅇ','ㅛ'],['']]
    #,[['ㅅ','ㅔ'],['ㅝ'],['ㅘ']]
]

NP = [
    
    [['ㅈ','ㅓ','ㄴ'],['ㄴ','ㅏ','ㄴ']],
    [['ㅈ','ㅓ'],['ㄴ','ㅏ']]
    
]

# 보조사
JX = [
    [['ㅇ','ㅣ','ㅇ','ㅛ'],['ㅇ','ㅣ','ㅇ','ㅑ']],
    [['ㅇ','ㅛ'],['']]
]

VX = [
    
    [['ㅈ','ㅜ'],['ㅈ','ㅜ'],['ㅈ','ㅝ']]
    
]

VV = [
    
    [['ㅇ','ㅗ'],['ㅇ','ㅗ'],['ㅇ','ㅘ']],
    [['ㅈ','ㅜ'],['ㅈ','ㅜ'],['ㅈ','ㅝ']],
    [['ㅎ','ㅏ'],['ㅎ','ㅏ'],['ㅎ','ㅐ']]
    
]

XR = [
    [['ㅈ','ㅚ','ㅅ','ㅗ','ㅇ'],['ㅁ','ㅣ','ㅇ','ㅏ','ㄴ'] ]
]

NNG = [
    [['ㅈ','ㅏ','ㅇ','ㅛ'],['ㅈ','ㅏ']]
]

#Not Use
EXC = [
    
    ['ㅜ',1],
    ['ㅗ',2],
    ['ㅏ',3]
    
]

EXC_word = [
    
    ['ㅅㅔ',2]
    ,['ㅅㅣㅂ',2]
    
]

EXC_deal = [
    
    ['ㅅㅔㅇㅛ', ['ㅏ','ㅣ'],['ㅜ','ㅓ'],['ㅡ','ㅓ']]
    
]

EXC_tags = [
    
    'EP',
    'EF'
    
]

####End of high -> low Dictionary####


#### low -> high Dictionary####

EF_ONLY_4S = [
    
    [['ㄷ','ㅏ'],['ㅅ','ㅡ','ㅂ','ㄴ','ㅣ','ㄷ','ㅏ'],['special']]
    
]

EF_ONLY_4C = [
    
    [['ㄷ','ㅏ'],['ㅅ','ㅡ','ㅂ','ㄴ','ㅣ','ㄷ','ㅏ']]
    ,[['ㅇ','ㅓ'],['ㅅ','ㅡ','ㅂ','ㄴ','ㅣ','ㄷ','ㅏ']]
    
]

VCP_EF_4 = [
    
    [['ㄷ','ㅏ'],['ㅇ','ㅣ','ㅂ','ㄴ','ㅣ','ㄷ','ㅏ']]
    
]

EF_AFTER_VCP_4 = [
    
    [['ㄷ','ㅏ'],['ㅂ','ㄴ','ㅣ','ㄷ','ㅏ']]
    
]

#### End of low -> high Dictionary####

def unite(input, dict):
    for i in dict:
        input = re.sub(i[0],i[1],input)
    return input
    
## 자모 단위로 문장을 나누고 합칠 때 쓰는 class ##
class Jamodealer:
    jamo = []
    pp = ''
    #각 단어들을 받아와서 자모단위로 나눈다.
    def __init__(self,lis_word):
    
        self.jamo = []
        for i in lis_word:
            self.jamo.append(split_syllables(i))
    
    ##사전에서 변환된 자모단위로 분리된 문장을 합칠 때 쓰는 함수이다.     
    def make_one(self):
        #list 형태로 저장된 자모들의 집합을 하나의 string pp에 저장한다. 
        self.pp = ''
        for i in self.jamo:
             self.pp= self.pp+i
        ##종성과 종성을 합쳐야 하는 경우가 있다면 합친다.        
        self.pp = unite(self.pp, con_dict)
        
        #자모 단위의 string에서 자모 단위로 사전을 만들고 거기에 index를 부여한다.        
        chars = list(set(self.pp))
        char_to_ix = { ch:i for i,ch in enumerate(chars) }
        ix_to_char = { i:ch for i,ch in enumerate(chars) }
        
        #자모 단위로 분리되었던 문장을 다시 하나로 합친다.
        jamo_numbers = [char_to_ix[x] for x in self.pp]
        restored_jamo = ''.join([ix_to_char[x] for x in jamo_numbers])
        #합쳐진 문장을 return 한다.
        restored_text = join_jamos(restored_jamo)
        return restored_text
##End of Jamodealer class##

def tojamo(korean_word):
    r_lst = []
    for w in list(korean_word.strip()):
        ## 영어인 경우 구분해서 작성함. 
        if '가'<=w<='힣':
            ## 588개 마다 초성이 바뀜. 
            ch1 = (ord(w) - ord('가'))//588
            ## 중성은 총 28가지 종류
            ch2 = ((ord(w) - ord('가')) - (588*ch1)) // 28
            ch3 = (ord(w) - ord('가')) - (588*ch1) - 28*ch2
            r_lst.append([CHOSUNG_LIST[ch1], JUNGSUNG_LIST[ch2], JONGSUNG_LIST[ch3]])
        else:
            r_lst.append([w])
    return r_lst

##Not Use
# def to1dim(input):
#     result=[]
#     for i in input:
#         for j in i:
#             result.append(j)
#     return result

##Not Use
# def to2dim(input):
#     result = []
#     li = []
#     for i in input:
#         if i == ' ':
#             result.append([' '])
#         else:
#             li.append(i)
#         if len(li)==3:
#             result.append(li)
#             li = []
#     return result

##Not Use
# def makeone(input):
#     result = ''
#     li = ''
#     for i in input:
#         if i[0]==' ':
#             result = result+' '
#         else:
#             ind = ord('가')
#             ind +=CHOSUNG_LIST.index(i[0])*588
#             ind +=JUNGSUNG_LIST.index(i[1])*28
#             ind +=JONGSUNG_LIST.index(i[2])
#             result = result+chr(ind)
#     return result

#Use in makestrdict
def li2str(input):
    st = ""
    for i in input:
        st = st+i
    return st

##Not Use
# def str2li(input):
#     li = []
#     for i in range(len(input)):
#         li.append(input[i])
#     return li

##Not Use
# def makejamodict(input):
#     result = []
#     for i in input:
#         bullet = []
#         one = []
#         two = []
#         gre1 = tojamo(i[0])
#         for j in gre1:
#             for k in j:
#                 one.append(k)
#         bullet.append(one)
#         gre2 = tojamo(i[1])
#         for j in gre2:
#             for k in j:
#                 two.append(k)
#         bullet.append(two)
#         result.append(bullet)
#     return result

##Used when make dictionaries
def makestrdict(input):
    result = []
    for i in input:
        bullet = []
        for j in range(len(i)):
            gre = li2str(i[j])
            bullet.append(gre)
        result.append(bullet)
    return result

#추가한 사전에 대한 str 사전을 생성, mapping 시키는 부분
#EP_dict = makestrdict(EP)
EF_dict = makestrdict(EF)
NP_dict = makestrdict(NP)
JX_dict = makestrdict(JX)
VX_dict = makestrdict(VX)
VV_dict = makestrdict(VV)
XR_dict = makestrdict(XR)
NNG_dict = makestrdict(NNG)

Dict_list=['EF','NP','JX','VX','VV','XR','NNG']

Dict_map = [EF_dict,NP_dict,JX_dict,VX_dict, VV_dict, XR_dict, NNG_dict]

EF_ONLY_dict = makestrdict(EF_ONLY)
EP_EF_dict = makestrdict(EP_EF)
VCP_EF_dict = makestrdict(VCP_EF)
A_EF_dict = makestrdict(A_EF)
XSA_EF_dict = makestrdict(XSA_EF)
XSV_EF_dict = makestrdict(XSV_EF)

#여기까지

#str dict의 low->high 
EF_ONLY_4S_dict = makestrdict(EF_ONLY_4S)
EF_ONLY_4C_dict = makestrdict(EF_ONLY_4C)
EF_AFTER_VCP_4_dict = makestrdict(EF_AFTER_VCP_4)
VCP_EF_4_dict = makestrdict(VCP_EF_4)

##Use in Changer
def to2lists(input):
    lis_word = []
    lis_tag = []
    #data = han.pos(input,ntags=22,flatten=True, join=False)
    data = mec.pos(input)
    for i in data:
        lis_word.append(i[0])
        lis_tag.append(i[1])
    return lis_word, lis_tag

#add in 2021.09.26

def indee(lis, input):
    
    rlis = []
    
    for i in range(len(lis)):
        if lis[i]==input:
            rlis.append(i)
            
    for i in range(len(rlis)):
        rlis[i] = rlis[i]-i
            
    return rlis

def ind_lili(lis_space,lis_lis):
    
    rlis = []
    k=0
    for i in range(len(lis_lis)):
        
        if k in lis_space:
            rlis.append(i)
            
        k = k+len(lis_lis[i])
        
    
    return rlis

def union(lis, lis_lis):
    
    k = 0
    for i in lis:
        lis_lis.insert(i+k,' ')
        k = k+1

## Main Class ## 
class Changer(object):
    ## High -> Low ##    
    def high_low(self, stc):
        result = stc
        
        space_list = indee(stc,' ')
        
        lis_word, lis_tag = to2lists(result)
        space_location = ind_lili(space_list, lis_word)
        jam = Jamodealer(lis_word)
        lis = []
        key = -1
        for i in H_LIST:
            if i in lis_word[-2]:
                key = 1
        #if key>0:
        if key>0 or key<0:     
            for i in range(len(lis_tag)):
                res = jam.jamo[i]
                for k in range(len(Dict_list)):
                    dic = []
                    if 'EF' in lis_tag[i]:
                        if 'EF' == lis_tag[i]:
                            dic = EF_ONLY_dict
                        elif 'EP' in lis_tag[i]:
                            dic = EP_EF_dict
                        elif 'VCP' in lis_tag[i]:
                            dic = VCP_EF_dict
                        else:
                            dic = A_EF_dict
                        
                    elif Dict_list[k] in lis_tag[i]:
                        dic = Dict_map[Dict_list.index(Dict_list[k])]
                        #res = jam.jamo[i]
                    for j in range(len(dic)):
                        if self.isExcept(dic[j])==1:
                            #ind = self.indicator(i,jam.jamo,lis_tag,EXC_word, EXC_tags)
                            for q in range(len(EXC_deal)):
                                if dic[j][0]==EXC_deal[q][0]:
                                    for r in range(1,len(EXC_deal[q])):
                                        if jam.jamo[i-1][-1]==EXC_deal[q][r][0]:
                                            res = re.sub(dic[j][0],EXC_deal[q][r][1],res);
                            #res = re.sub(dic[j][0],dic[j][ind],res)
                        else:
                            #print('tt')
                            res = re.sub(dic[j][0],dic[j][1],res)
                            
                        #jam.jamo[i] = res
                lis.append(res)
            
            
            #print(jam.jamo[i])
        
        union(space_location, lis)
        jam.jamo = []
        for i in range(len(lis)):
            jam.jamo.append(lis[i])
            #print(lis[i])
        
        #union(space_location, jam.jamo)
        
        return jam.make_one()
    
    def low_high(self, stc):
        result = stc
        
        space_list = indee(stc, ' ')
        
        lis_word, lis_tag = to2lists(result)
        space_location = ind_lili(space_list, lis_word)
        jam = Jamodealer(lis_word)
        lis = []
        key = -1
        
        for i in H_LIST:
            if i not in lis_word[-2]:
                
                key = 1
        #if key>0:
        if key>0 or key<0:    
            res = jam.jamo[-2]
            dic = []
            if 'EF' in lis_tag[-2]:
                
                if 'EF' == lis_tag[-2]:
                    if '+' in lis_tag[-3]:
                        dic = EF_ONLY_4C_dict
                    elif lis_tag[-3]=='VCP':
                        dic = EF_AFTER_VCP_4_dict
                    else:
                        dic = EF_ONLY_4S_dict
                elif 'EP' in lis_tag[-2]:
                    dic = EP_EF_dict
                elif 'VCP' in lis_tag[-2]:
                    dic = VCP_EF_4_dict
                else:
                    dic = A_EF_dict
            for i in range(len(dic)):
                if len(dic[i])>2:
                    res = re.sub(dic[i][0],dic[i][1],res)
                else:
                    res = re.sub(dic[i][0],dic[i][1],res)
            jam.jamo[-2] = res
            
        union(space_location, jam.jamo)
        
        return jam.make_one()
            
        
        
        
    def isExcept(self, input):
        if input[1]=='special':
            return 1
        else:
            return 0
        
    def indicator(self, ind, lis, tag, ex_word, exc_tags):
        re = 1
        for j in range(len(ex_word)):
            if exc_tags[j] in tag[ind+1]:
                for i in range(len(ex_word)):
                    if ex_word[i][0] in lis[ind+1]:
                       
                        re = ex_word[i][1]
                        break
                        
                        print(lis[ind-1])
        return re
        
        
    def processText(self,stc):
        result = stc
        res = self.high_low(result)
        #spacing = Spacing()
        #res = spacing(res)
        return res
    
    def processText_0(self,stc):
        result = stc
        res = self.low_high(result)
        return res
    

In [2]:
#txt = '전 그것이 맞다고 생각합니다.'
tx = '밥 먹으세요.'
ch = Changer()
#tt = ch.processText(txt)
ttt = ch.processText(tx)
#print(tt)
print(ttt)

밥 먹어.


In [3]:
number = 1858355
texts = pd.read_table('./data/hgu_clean.kr', sep ='\n')

new_data = {
    
    '26일이요?':'26일이요?'
    
}

idx = 0

temp1 = texts[texts.index < idx]
temp2 = texts[texts.index>= idx]

texts = temp1.append(new_data,ignore_index=True).append(temp2, ignore_index=True)

texts = texts['26일이요?']

num = len(texts)

ch = Changer()
f = open('./hgu_clean.low.kr', 'w',encoding = 'utf-8')
for i in range(0,num):
    if texts[i][-1] !='?'and texts[i][-1] !='.' and texts[i][-1] !='!':
        texts[i] = texts[i]+'.'
    tt = ch.processText(texts[i])
    f.write(tt)
    f.write('\n')
f.close()

In [22]:
#import pandas as pd

texts1 = pd.read_table('./data/hgu_clean.kr.shuf', sep ='\n')

new_data = {
    
    '그것은 가솔린 수소 또는 전기 모터로 움직일 수 있다':'그것은 가솔린 수소 또는 전기 모터로 움직일 수 있다'
    
}

idx = 0

temp1 = texts1[texts1.index < idx]
temp2 = texts1[texts1.index>= idx]

texts1 = temp1.append(new_data,ignore_index=True).append(temp2, ignore_index=True)

texts1 = texts1['그것은 가솔린 수소 또는 전기 모터로 움직일 수 있다']

num = len(texts1)
a = len(texts1)
#print(a)
ch = Changer()
f = open('./hgu_clean.low.kr.shuf', 'w',encoding = 'utf-8')
for i in range(0,num):
    if texts1[i][-1] !='?'and texts1[i][-1] !='.' and texts1[i][-1] !='!':
        texts1[i] = texts1[i]+'.'
    tt = ch.processText(texts1[i])
    f.write(tt)
    f.write('\n')
f.close()

KeyError: 0